## Goals: Hyper Parameter Optimisation of *QRF* model

This notebook propose different methods of hyper parameter optimisation based on X-Validation :
* Random Search
* Genetic algorithm

# 1. Data Import and Setup

Imports necessary libraries, sets up environment paths.

In [ ]:
# Standard library imports
import os
import sys

# Third-party imports
from functools import partial
import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from quantile_forest import RandomForestQuantileRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SelectKBest, f_regression


# Append project root to sys.path for local imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))

# Local application imports
from src.utils.model import split_dataset, get_station_stats, custom_log_likelihood
from src.utils.SpatioTemporalSplit import SpatioTemporalSplit

Defines constants :
* INPUT_DIR must be the same as the one defined in *00 Preprocessing/Feature Engineering*.
* MODEL_DIR is the directory where the exploration models will be saved.

In [ ]:
INPUT_DIR = "../../../../data/input/"
MODEL_DIR = "../../../../models/exploration/"

SEED = 42 
ALPHA = 0.1
WEEK_TO_PREDICT=1 

# columns to drop : target at different horizon, station_code, and features removed from Feature Selection
TO_DROP = ["water_flow_week1", "station_code", "water_flow_week2", "water_flow_week3", "water_flow_week4"]

# 2. Data Loading
Load in the baseline datasets, create the directory to save models.

In [ ]:
# load the dataset
ds_train = pd.read_csv(f"{INPUT_DIR}dataset_baseline.csv")
train_data = ds_train.copy()
train_data.reset_index(inplace=True)
train_data = train_data.loc[:, ~train_data.columns.duplicated()]
ds_train = ds_train.set_index("ObsDate")

In [ ]:
y_train = train_data[f"water_flow_week{WEEK_TO_PREDICT}"]

In [ ]:
# 1. Prepare the data
cv_data = train_data.copy()

station_stats = get_station_stats(
    y_train.to_numpy(),
    train_data["station_code"].to_numpy()
)

# 2. Define the predictor columns.
cols_to_drop = TO_DROP.copy()
cols_to_drop += ["ObsDate"]

predictor_cols = [col for col in cv_data.columns if col not in cols_to_drop]

# 3. Create a ColumnTransformer to select the predictor columns.
preprocessor = ColumnTransformer(transformers=[
    ('select', 'passthrough', predictor_cols)
], remainder='drop')

# 4. Instantiate your model.
qrf_week1 = RandomForestQuantileRegressor(n_estimators=10, max_depth=10, min_samples_leaf=10)

# 5. Build the pipeline.
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])
scorer = partial(custom_log_likelihood,
                 cv_data=cv_data,
                 station_stats=station_stats,
                 alpha=ALPHA)


# 7. Define the custom spatio-temporal cross-validator.
cv = SpatioTemporalSplit(
    n_splits=10,
    date_col='ObsDate',
    station_col='station_code',
    temporal_frac=0.75,
    spatial_frac=0.75,
    random_state=42
)

# 8. Define the hyperparameter distributions for random search.
param_distributions = {
    'model__n_estimators': [5, 10, 15],
    'model__max_depth': [5, 6, 7],
    'model__min_samples_leaf': [1, 2, 5]
}

# 9. Set up RandomizedSearchCV.
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=20,            # Number of parameter settings sampled
    scoring=scorer,       # Use our custom scorer
    cv=cv,                # Our custom spatio-temporal splitter
    random_state=42,
    n_jobs=-1             # Use all available cores
)

# 10. Run the random search on your data.
random_search.fit(cv_data, y_train)

In [ ]:
# 11. Print the best parameters and best score.
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)